In [9]:
import pandas as pd
import datetime
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from pandas.plotting import lag_plot
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [10]:
Data = pd.read_csv("Website Vistiors Daywise - Sheet1.csv")
pd.set_option('display.max_rows', None)
Data

FileNotFoundError: ignored

In [ ]:
#changing from dd-mm-yyyy to yyyy-mm-dd
import datetime
date = list()
for d in Data.Date.tolist():
    t = datetime.datetime.strptime(d, "%d-%m-%Y").strftime("%Y-%m-%d")
    date.append(t)

Data['date'] = date
Data.drop("Date", axis =1, inplace =True)
Data

# EDA and data visualization

In [ ]:
Data.info()

In [ ]:
# changing the type of "Date" column from object to datetime
Data["date"]= pd.to_datetime(Data["date"])

check for duplicate rows

In [ ]:
duplicate = Data[Data.duplicated()]

print("Duplicate Rows :")

In [ ]:
Data.describe().T

In [ ]:
Data.columns = ["visitors","date"]
df = Data.set_index("date")

In [ ]:
plt.figure(figsize = (8,4))
plt.subplot(211)
sns.boxplot(df.visitors)
plt.subplot(212)
sns.distplot(df.visitors)
plt.show()

In [ ]:
df.loc[df['visitors']>=4500]

In [ ]:
#repacing the outliers with median

print("median :", df['visitors'].quantile(0.50))
print(df['visitors'].quantile(0.95))
df['visitors'] = np.where(df['visitors'] > 4500, 2751, df['visitors'])
plt.figure(figsize = (8,4))
plt.subplot(211)
sns.boxplot(df.visitors)
plt.subplot(212)
sns.distplot(df.visitors)
plt.show()

In [ ]:
df

In [ ]:
df['visitors'].plot(figsize=(8, 3))

plt.ylabel("visitors")
plt.show()

In [ ]:
df.isnull().sum()

In [ ]:
df['month'] = df.index.month
df['day'] = df.index.day
df['year'] = df.index.year
df

In [ ]:
# if the day is a weekend or weekday?
from datetime import datetime
import calendar
def weekend_or_weekday(year,month,day):

    d = datetime(year,month,day)
    if d.weekday()>4:
        return 1
    else:
        return 0

df['weekend'] = df.apply(lambda x:weekend_or_weekday(x['year'], x['month'], x['day']), axis=1)
df.head()

In [ ]:
#comparing internet traffic on weekend and working day
plt.figure(figsize = (3,3))
sns.boxplot(x="weekend",y="visitors",data=df)
plt.title(" internet traffic on a weekend or not weekend")
plt.show()

In [ ]:
# if the day is a holiday or not?
from datetime import date
import holidays

def is_holiday(x):

    india_holidays = holidays.country_holidays('IN')

    if india_holidays.get(x):
        return 1
    else:
        return 0

df['holidays'] = df.index.to_series().apply(is_holiday)
df.head()

In [ ]:
# comparing internet traffic on holidays and non-holidays
plt.figure(figsize = (3,3))
plt.title(" internet traffic on a holiday or not a holiday")
sns.boxplot(x="holidays",y="visitors",data=df)
plt.show()

In [ ]:
# Boxplots
plt.figure(figsize=(8,6))
plt.subplot(211)
sns.boxplot(x="month",y="visitors",data=df,order=[10,11,12,1,2,3])
plt.subplot(212)
sns.boxplot(x="year",y="visitors",data=df)
plt.subplots_adjust(left=0.1,bottom=0.1, right=0.9,top=0.9,wspace=0.2,hspace=0.4)
plt.show()

In [ ]:
figure, axes = plt.subplots(2, 3, figsize=(10, 5))
sns.lineplot(ax=axes[0, 0], data = df.loc[df["month"]==10].sort_values(by='day'), x='day',y='visitors')
axes[0,0].legend(["october"])
sns.lineplot(ax=axes[0, 1], data = df.loc[df["month"]== 11].sort_values(by='day'), x='day',y='visitors')
sns.lineplot(ax=axes[0, 2], data = df.loc[df["month"]==12].sort_values(by='day'), x='day',y='visitors')
sns.lineplot(ax=axes[1, 0], data = df.loc[df["month"]==1].sort_values(by='day'), x='day',y='visitors')
sns.lineplot(ax=axes[1, 1], data = df.loc[df["month"]==2].sort_values(by='day'), x='day',y='visitors')
sns.lineplot(ax=axes[1, 2], data = df.loc[df["month"]==3].sort_values(by='day'), x='day',y='visitors')
axes[0,1].legend(["november"])
axes[0,2].legend(["december"])
axes[1,0].legend(["january"])
axes[1,1].legend(["february"])
axes[1,2].legend(["march"])
plt.subplots_adjust(left=0.1,bottom=0.1, right=0.9,top=0.9,wspace=0.2,hspace=0.4)
plt.show()

In [ ]:
med = df.loc[df['month']==2,'visitors'].quantile(0.50)
upper_limit = df.loc[df['month']==2,'visitors'].quantile(0.95)
print('median : ', med)
print('upper limit : ', upper_limit)

In [ ]:
#repacing the outliers in feb with median
med = df.loc[df['month']==2,'visitors'].quantile(0.50)
upper_limit = df.loc[df['month']==2,'visitors'].quantile(0.95)
df.loc[df['month']==2,'visitors'] = np.where(df.loc[df['month']==2,'visitors'] > upper_limit, med,
                                             df.loc[df['month']==2,'visitors'])



In [ ]:
plt.figure(figsize=(10,5))
for i in [1,10,14,50]:
    df["visitors"].rolling(i).mean().plot(label=str(i))

plt.legend(loc=2)
plt.show()

In [ ]:
ts_mul = seasonal_decompose(df.visitors,model="multiplicative", period =50)
fig = ts_mul.plot()
plt.show()

In [ ]:
ts_mul = seasonal_decompose(df.visitors,model="additive", period =50)
fig = ts_mul.plot()
plt.show()

In [ ]:
# Tail-rolling average transform
rolling = df.visitors.rolling(window=2)
rolling_mean = rolling.mean()
print(rolling_mean.head())
# plot original and transformed dataset
plt.figure(figsize = (8,4))
df.visitors.plot()
rolling_mean.plot(color='red')
plt.legend(['actual time series','transformed time series'])
plt.ylabel("no. of visitors")
plt.xlabel("observation")
plt.show()
df1 = pd.DataFrame(rolling_mean.dropna())
df1.head()

In [ ]:
plt.figure(figsize = (8,3))
axes = lag_plot(df.visitors)
lag_plot(df1.visitors,ax=axes, c = 'orange' )
plt.legend(['Raw data', 'Tail-rolling average(window=2)'])
plt.show()

**A lag plot checks whether a data set or time series is random or not. Random data should not exhibit any identifiable structure in the lag plot. Non-random structure in the lag plot indicates that the underlying data are not random.**

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
plot_acf(df1.visitors,lags=100,ax = ax[0])
plot_pacf(df1.visitors,lags=50, ax = ax[1])
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.3,
                    hspace=0.4)
plt.show()

In [ ]:
# Dickey_fuller test to check if the series is stationary or non-stationary

from statsmodels.tsa.stattools import adfuller
res = adfuller(df1.visitors,autolag='AIC')
print('Augmneted Dickey_fuller Statistic: %f' % res[0])
print('p-value: %f' % res[1])
print('Lags: %f' % res[2])
# printing the critical values at different alpha levels.
print('critical values at different levels:')
for k, v in res[4].items():
    print('\t%s: %.3f' % (k, v))
print(f'Result: The series is {"not " if res[1] > 0.05 else ""}stationary')

In [ ]:
#Kwiatkowski-Phillips-Schmidt-Shin(KPSS) test to check if the series is trend stationary
from statsmodels.tsa.stattools import kpss
def kpss_test(series, **kw):
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')

kpss_test(df1, regression="ct", nlags="auto")

**the series has both trend and seasonality and this is why showing not stationary in ADF test but not stationary around the deterministic tred in the KPSS test.**

# Preparing Dataset for modelling

In [ ]:
from numpy import log
df1["log_visitors"] = log(df1["visitors"])
df1['t'] = range(1,(len(df1["visitors"])+1))
df1['t_sq'] = df1['t']**2
df1['month'] = df1.index.month
df1['day'] = df1.index.day
df1

In [ ]:
#splitting the time series obtained after transformation
X = df1.visitors
train, test = np.split(X, [int(.67 *len(X))])
print("train dimensions", train.shape)
print("test dimensions", test.shape)

# Model building

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
#Quadratic
#splitting the data
train1, test1 = np.split(df1, [int(.67 *len(df1))])
Quad = smf.ols('visitors~t+t_sq',data=train1).fit()
pred_Quad = pd.Series(Quad.predict(test1[["t","t_sq"]]))
rmse_Quad = np.sqrt(np.mean((np.array(test1['visitors'])-np.array(pred_Quad))**2))
print(rmse_Quad)
plt.figure(figsize = (6,3))
pred_Quad.plot()
test1['visitors'].plot()
plt.legend(['predicted', 'actual'],loc = (0.75,0))
plt.show()

In [ ]:
#Exponential

Exp = smf.ols('log_visitors~t',data=train1).fit()
pred_Exp = pd.Series(Exp.predict(test1[["t","t_sq"]]))
rmse_Exp = np.sqrt(np.mean((np.array(test1['visitors'])-np.array(pred_Exp))**2))
print(rmse_Exp)
plt.figure(figsize = (6,3))
pred_Exp.plot()
test1['visitors'].plot()
plt.legend(['predicted', 'actual'],loc = (0.75,0))
plt.show()

In [ ]:
#Multiplicative Additive Seasonality
Mul_Add_sea = smf.ols('log_visitors~t+t_sq+month+day',data = train1).fit()
pred_Mult_add_sea = pd.Series(Mul_Add_sea.predict(test1))
rmse_Mult_add_sea = np.sqrt(np.mean((np.array(test1['visitors'])-np.array(pred_Mult_add_sea))**2))
print(rmse_Mult_add_sea)
plt.figure(figsize = (6,3))
pred_Mult_add_sea.plot()
test1['visitors'].plot()
plt.legend(['predicted', 'actual'],loc = (0.75,0))
plt.show()

In [ ]:
# Holt method
from statsmodels.tsa.holtwinters import Holt
hw_model = Holt(train).fit(smoothing_level=0.8, smoothing_trend=0.2)
pred_hw = hw_model.predict(start = test.index[0],end = test.index[-1])
rmse_hw = np.sqrt(np.abs((test-pred_hw)**2)).mean()
print("rmse:", rmse_hw)
plt.figure(figsize = (6,3))
pred_hw.plot()
test.plot()
plt.legend(['predicted', 'actual'],loc = (0.75,0))
plt.show()


In [ ]:
# Holts winter exponential smoothing with additive seasonality and additive trend
from statsmodels.tsa.holtwinters import ExponentialSmoothing
hwe_model_add_add = ExponentialSmoothing(train,seasonal="add",trend="add",seasonal_periods=21).fit() #add the trend to the model
pred_hwe_add_add = hwe_model_add_add.predict(start = test.index[0],end = test.index[-1])
rmse_hw_add_add = np.sqrt(np.abs((test-pred_hwe_add_add)**2)).mean()
print("rmse:", rmse_hw_add_add)
plt.figure(figsize = (6,3))
pred_hwe_add_add.plot()
test.plot()
plt.legend(['predicted', 'actual'],loc = (0.75,0))
plt.show()

In [ ]:
#Holts winter exponential smoothing with multiplicative seasonality and additive trend
hwe_model_mul_add = ExponentialSmoothing(train,seasonal="mul",trend="add",seasonal_periods=21).fit()
pred_hwe_mul_add = hwe_model_mul_add.predict(start = test.index[0],end = test.index[-1])
rmse_hw_mul_add = np.sqrt(np.abs((test-pred_hwe_mul_add)**2)).mean()
print("rmse:", rmse_hw_mul_add)
plt.figure(figsize = (6,3))
pred_hwe_mul_add.plot()
test.plot()
plt.legend(['predicted', 'actual'])
plt.show()

In [ ]:
# Auto regression model
from statsmodels.tsa.ar_model import AutoReg
from numpy import sqrt
# contrived dataset
# fit model
model = AutoReg(train, lags=18)
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(start = test.index[0],end = test.index[-1], dynamic=False)
rmse_AR = np.sqrt(np.abs((test-yhat)**2)).mean()
print("rmse:", rmse_AR)
plt.figure(figsize = (6,3))
yhat.plot()
test.plot()
plt.legend(['predicted', 'actual'])
plt.show()

# ARIMA: Auto Regressive Integrated Moving Average model


In [ ]:
# evaluate an ARIMA model for a given order (p,d,q) and return RMSE
def evaluate_arima_model(X, arima_order):
# prepare training dataset
    history = [x for x in train]
    predictions = []
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
# calculate out of sample error
    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse

In [ ]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float('inf'), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(train, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                        print('ARIMA%s RMSE=%.3f' % (order,rmse))
                except:
                    continue
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))

# evaluate parameters
p_values = range(1, 5)
d_values = range(0, 5)
q_values = range(0, 5)
evaluate_models(train1, p_values, d_values, q_values)

In [ ]:
#ARIMA (rolling forcast ARIMA model)
history = [n for n in train]
predictions = []
future_steps = len(test)
#walk forward validation
for t in range(len(test)):
    model = ARIMA(history, order=(3,0,4))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    history.append(output[0])
rmse_rolling_ARIMA = sqrt(mean_squared_error(test, predictions))
print(rmse_arima)
plt.plot(test.values)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
#SARIMA model
smodel = pm.auto_arima(X, start_p=1, start_q=1,
                         test='adf',
                         max_p=5, max_q=5, m=7,
                         start_P=0, seasonal=True,
                         d=None, D=1, trace=True,
                         error_action='ignore',
                         suppress_warnings=True,
                         stepwise=True)

smodel.summary()

In [ ]:
import statsmodels.api as sm
# rolling sarima forecast
x = X.values
size = int(len(x) * 0.66)
train, test = x[0:size], x[size:len(x)]
history = [n for n in train]
predictions = list()
future_steps = len(test)
# ARIMA with walk-forward validation
for t in range(future_steps):
    model = model=sm.tsa.statespace.SARIMAX(history, order=(3,0,0), seasonal_order=(1,1,2,7))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    history.append(output[0])

# evaluate forecasts
rmse_rollong_SARIMA = np.sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse_rollong_SARIMA)

# make as panda series
fc_series = pd.Series(predictions, index=list(range(len(train), len(history))))
test_series = pd.Series(test, index=list(range(len(train), len(history))))

# plot forecasts against actual outcomes
plt.figure(figsize = (10,4))
plt.plot(train,color='black')
plt.plot(test_series)
plt.plot(fc_series, color='red')
plt.legend(['train','test','predicted'])
plt.show()

# LSTM forcasting

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
size = int(len(df1) * 0.70)
X=df1[['visitors']]
train, test = X.iloc[0:size], X.iloc[size:len(X)]

In [ ]:
def forecast(model, history, n_input):
    data = np.array(history)
    # retrieve last observations for input data
    input_x = data[-n_input:,0]
    # reshape into [1, n_input, 1]
    input_x = input_x.reshape((1, len(input_x), 1))
    # forecast the next week
    yhat = model.predict(input_x, verbose=0)
    # we only want the vector forecast
    yhat = yhat[0]
    return yhat

In [ ]:
n_input=20
n_features = 1
generator = TimeseriesGenerator(train.values,
                                train.values,
                                length=n_input,
                                batch_size=1)
# define model
model = Sequential()
model.add(LSTM(100,activation="relu",input_shape=(n_input, n_features)))
model.add(Dense(1),)
model.compile(optimizer='adam', loss='mse')
model.summary()
# fit model
model.fit_generator(generator, steps_per_epoch=1, epochs=200, verbose=0)

# make a forecast
history = [n for n in train.values]
predictions = list()
for i in range(len(test.values)):
    yhat_sequence = forecast(model, history, n_input)
    # store the predictions
    predictions.append(yhat_sequence[0])
    # get the forecatsed value and add to history for next day forcast
    history.append(yhat_sequence)

res_LSTM = pd.DataFrame(test.values)
res_LSTM["predicted"] = predictions
res_LSTM.columns = ["actual","predicted"]
rmse_LSTM = sqrt(mean_squared_error(res_LSTM["actual"], res_LSTM["predicted"]))
print("rmse :",rmse_LSTM)
plt.figure(figsize = (6,3))
res_LSTM["actual"].plot()
res_LSTM["predicted"].plot()
plt.legend(['actual', 'predicted'])
plt.show()

# Model evaluation

In [ ]:
model_evaluation = {'Quadratic':rmse_Quad,'exponential':rmse_Exp,'hw': rmse_hw,'hw_add_add':rmse_hw_add_add,'hw_mul_add':rmse_hw_mul_add,'auto_regression':rmse_AR,
                    'rolling_ARIMA':rmse_rolling_ARIMA,'rolling_SARIMA': rmse_rollong_SARIMA,'LSTM':rmse_LSTM}
results = pd.DataFrame(data = model_evaluation, index = range(1)).T.reset_index()
results.columns = ['model','RMSE']
results = results.sort_values(by='RMSE')
plt.figure(figsize=(8,3))
ax = sns.barplot(x = results.model,y=results.RMSE, data=results)
plt.xticks(rotation = 45)
plt.ylim(0,3200)
for i in ax.containers:
    ax.bar_label(i,)

# Final model